In [65]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import pandas as pd

In [3]:
?pipeline


In [66]:
model_dir = "/home/liubinxu/下载/FinBERT_L-12_H-768_A-12_pytorch/"

In [67]:
finbert = BertForSequenceClassification.from_pretrained(model_dir)

Some weights of the model checkpoint at /home/liubinxu/下载/FinBERT_L-12_H-768_A-12_pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [68]:
tokenizer = BertTokenizer.from_pretrained(model_dir)

In [19]:
tokenizer


BertTokenizer(name_or_path='/home/liubinxu/下载/FinBERT_L-12_H-768_A-12_pytorch/', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [69]:
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [29]:
sentences = ["明天大涨",  
             "大跌， 风险将至"]
results = nlp(sentences)

In [30]:
results

[{'label': 'LABEL_1', 'score': 0.6478438973426819},
 {'label': 'LABEL_1', 'score': 0.5857692956924438}]

In [33]:
easy_money2 = pd.read_table("../easymoney_data/002.clean.text")

In [36]:
easy_money2 = easy_money2.rename(columns={"阅读": "read_num", "评论": "commit_num", "标题": "title", "最后更新": "date"})

In [44]:
choose = list(easy_money2.title)[-20:]

In [70]:
choose

['即使生活再不如意，新的一年到了，祝所有的股友们新年',
 '过去五年，IPO数量和融资规模分别为：2017年438家，融',
 '股市是最跟随自然节奏的，最不顺应自然的地方，所以股',
 ' 希望2023年水平能更进一步，我炒股满两年的时间了，可',
 '2003',
 '2o22年走完最后一步。2023年迈开第一步。喜迎开门大吉',
 ' 新年快乐！加油～[爱心][献花]',
 '今天要过去了，今年也要过去了。',
 ' 天使投资人郭涛接受《华夏时报》记者采访表示，受俄乌',
 '2023许个愿望吧，我先来，1祝家人们身体健康，万事如',
 '股神附体[大笑]',
 '新年快乐老铁们',
 '2023收益翻十倍',
 ' 跨年夜告诉大家一个好消息一个坏消息！2021年和2021年',
 '新年快乐！祝大家新的一年能回本！并赚钱！',
 ' 新年快乐，愿各位今年回本的回本，翻倍的翻倍，爆赚！',
 '新年快乐，新的一年继续薅主力的羊毛。',
 '各位股市老铁，兔年来了。祝老铁们兔年吉祥如意，个个',
 '大盘会涨吗？',
 '2023发发发']

In [72]:
nlp(list(choose))

[{'label': 'LABEL_0', 'score': 0.5698588490486145},
 {'label': 'LABEL_0', 'score': 0.6250249743461609},
 {'label': 'LABEL_0', 'score': 0.5677523612976074},
 {'label': 'LABEL_1', 'score': 0.5289913415908813},
 {'label': 'LABEL_0', 'score': 0.5779617428779602},
 {'label': 'LABEL_0', 'score': 0.5910698175430298},
 {'label': 'LABEL_1', 'score': 0.517271101474762},
 {'label': 'LABEL_0', 'score': 0.5909019112586975},
 {'label': 'LABEL_0', 'score': 0.5499157309532166},
 {'label': 'LABEL_0', 'score': 0.5286133289337158},
 {'label': 'LABEL_0', 'score': 0.5200566649436951},
 {'label': 'LABEL_0', 'score': 0.5232834219932556},
 {'label': 'LABEL_0', 'score': 0.5761458873748779},
 {'label': 'LABEL_0', 'score': 0.5734653472900391},
 {'label': 'LABEL_0', 'score': 0.528021514415741},
 {'label': 'LABEL_0', 'score': 0.5198492407798767},
 {'label': 'LABEL_0', 'score': 0.5467403531074524},
 {'label': 'LABEL_1', 'score': 0.502379834651947},
 {'label': 'LABEL_0', 'score': 0.5026551485061646},
 {'label': 'LAB

In [71]:
results = nlp(list(easy_money2.title))

KeyboardInterrupt: 

In [52]:
results_score_list = [x['score'] for x in results]

In [53]:
easy_money2["sentiments"] = results_score_list

In [54]:
easy_money2.to_csv("002.senti.tsv", sep="\t")

In [55]:
easy_money2["date"] = easy_money2["date"].map(lambda x: x.split(" ")[0])
nums = easy_money2["sentiments"].groupby(easy_money2["date"]).count()
sums = easy_money2["sentiments"].groupby(easy_money2["date"]).sum()
day_senti = pd.DataFrame()
day_senti["num"] = nums
day_senti["sum"] = sums
day_senti["senti_value"] = sums/nums
day_senti.to_csv("2023_easy_money_sens_bert")

In [49]:
choose3 = ["上证指数的走势被港股带回原有的反弹轨道，但随着港股今日回落，A股也跟着回落了。",

    "假期外围普涨，港股反弹创了新高，越过左肩高点，美股假期也是持续反弹，在特斯拉的带动下造车新势力普涨，因此今日A股汽车类个股全线反弹。",

    "A股始终就是跟着别人走，看别人脸色。",

    "现实中的二级市场远比车企以及某站车评人吹嘘来的现实的多。",

     "2月份整体是震荡的走势，也就是说指数要构筑一个中枢扩展的结构才能再上。"]

In [50]:
nlp(choose3)

[{'label': 'LABEL_1', 'score': 0.6562941074371338},
 {'label': 'LABEL_1', 'score': 0.6379492282867432},
 {'label': 'LABEL_1', 'score': 0.5668553709983826},
 {'label': 'LABEL_1', 'score': 0.5000359416007996},
 {'label': 'LABEL_1', 'score': 0.6239464282989502}]

In [56]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

NameError: name 'AutoTokenizer' is not defined

In [ ]:
#默认bert中文模型

In [57]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [59]:
nlp2 = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

The model 'BertForMaskedLM' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'EsmForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceCla

In [60]:
choose = list(easy_money2.title)[-20:]

In [61]:
choose

['即使生活再不如意，新的一年到了，祝所有的股友们新年',
 '过去五年，IPO数量和融资规模分别为：2017年438家，融',
 '股市是最跟随自然节奏的，最不顺应自然的地方，所以股',
 ' 希望2023年水平能更进一步，我炒股满两年的时间了，可',
 '2003',
 '2o22年走完最后一步。2023年迈开第一步。喜迎开门大吉',
 ' 新年快乐！加油～[爱心][献花]',
 '今天要过去了，今年也要过去了。',
 ' 天使投资人郭涛接受《华夏时报》记者采访表示，受俄乌',
 '2023许个愿望吧，我先来，1祝家人们身体健康，万事如',
 '股神附体[大笑]',
 '新年快乐老铁们',
 '2023收益翻十倍',
 ' 跨年夜告诉大家一个好消息一个坏消息！2021年和2021年',
 '新年快乐！祝大家新的一年能回本！并赚钱！',
 ' 新年快乐，愿各位今年回本的回本，翻倍的翻倍，爆赚！',
 '新年快乐，新的一年继续薅主力的羊毛。',
 '各位股市老铁，兔年来了。祝老铁们兔年吉祥如意，个个',
 '大盘会涨吗？',
 '2023发发发']

In [64]:
results2 = nlp2(choose
               )

KeyError: 106726

In [73]:
model_dir2 = "/liubinxu/liubinxu/finance/sentiment_cls/models/albert_chinese_small"

In [74]:
finbert2 = BertForSequenceClassification.from_pretrained(model_dir)

Some weights of the model checkpoint at /home/liubinxu/下载/FinBERT_L-12_H-768_A-12_pytorch/ were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [75]:
tokenizer2 = BertTokenizer.from_pretrained(model_dir2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [76]:
nlp2 = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

In [77]:
nlp2

In [83]:
nlp2(list(choose))

[{'label': 'LABEL_0', 'score': 0.5698588490486145},
 {'label': 'LABEL_0', 'score': 0.6250249743461609},
 {'label': 'LABEL_0', 'score': 0.5677523612976074},
 {'label': 'LABEL_1', 'score': 0.5289913415908813},
 {'label': 'LABEL_0', 'score': 0.5779617428779602},
 {'label': 'LABEL_0', 'score': 0.5910698175430298},
 {'label': 'LABEL_1', 'score': 0.517271101474762},
 {'label': 'LABEL_0', 'score': 0.5909019112586975},
 {'label': 'LABEL_0', 'score': 0.5499157309532166},
 {'label': 'LABEL_0', 'score': 0.5286133289337158},
 {'label': 'LABEL_0', 'score': 0.5200566649436951},
 {'label': 'LABEL_0', 'score': 0.5232834219932556},
 {'label': 'LABEL_0', 'score': 0.5761458873748779},
 {'label': 'LABEL_0', 'score': 0.5734653472900391},
 {'label': 'LABEL_0', 'score': 0.528021514415741},
 {'label': 'LABEL_0', 'score': 0.5198492407798767},
 {'label': 'LABEL_0', 'score': 0.5467403531074524},
 {'label': 'LABEL_1', 'score': 0.502379834651947},
 {'label': 'LABEL_0', 'score': 0.5026551485061646},
 {'label': 'LAB

In [80]:
data = "/liubinxu/liubinxu/finance/sentiment_cls/input/data01/test1"
with open(data, 'r') as f:
    data_list = f.readlines()
data_list = [x.strip() for x in data_list]

In [81]:
data_list

['仁东控股控股股东及一致行动人拟被动减持不超过公司股份\n',
 '青岛啤酒复星集团减持公司万股股占总股本约\n',
 '隆基股份李春安于月日至日累计减持公司股份\n',
 '芯源微两股东拟合计减持不超公司股份\n',
 '福耀玻璃河仁慈善基金会拟减持不超过万股公司股股份\n',
 '华大基因股东减持股份比例累计达到\n',
 '迪安诊断第一期员工持股计划股份出售完毕\n',
 '建研院股东冯国宝拟减持不超过公司股份\n',
 '西藏珠峰因信托计划到期西藏信托减持公司股份\n',
 '长阳科技陶春风及其一致行动人减持公司股份\n',
 '华谊嘉信杭州福石资产管理有限公司减持公司股份\n',
 '完美世界公司第一期员工持股计划存续期届满暨计划终止卖出约万股\n',
 '未名医药高宝林减持公司股份\n',
 '迪安诊断第一期员工持股计划股份出售完毕暨计划终止卖出万股\n',
 '特力远致富海减持公司股份\n',
 '驰宏锌锗苏庭宝减持公司股份\n',
 '上海莱士控股股东科瑞天诚投资控股有限公司及一致行动人被动减持公司股份\n',
 '宏发股份联发集团减持公司股份\n',
 '景峰医药股东叶湘武拟减持不超过公司股份\n',
 '西部超导深圳市创新投资集团有限公司减持股份\n',
 '震安科技广发信德减持公司股份\n',
 '星帅尔控股股东实际控制人楼月根减持公司股份\n',
 '荣晟环保控股股东实际控制人之一的冯荣华减持比例达\n',
 '天奈科技减持达到\n',
 '科林电气控股股东及一致行动人减持公司股份\n',
 '天创时尚香港高创减持公司股份\n',
 '威派格李纪玺及其一致行动人减持公司可转债\n',
 '星宇股份周晓萍减持公司可转债\n',
 '海航科技被问询要求说明公司资产置入及本次出售之间关系\n',
 '合众思壮收到关注函要求对比说明思壮北斗投资性房地产评估增值的合理性\n',
 '中国巨石决定终止筹划重大资产重组事项股票复牌\n',
 '中材科技决定终止筹划重大资产重组事项股票复牌\n',
 '中芯国际一份美国民事诉状将公司及部分董事列为被告将作出积极抗辩\n',
 '仁东控股控股股东及其一致行动人两融业务有触发强制平仓的可能性\n',
 '仁东控股京基集团在涨停板减持约公司股份\n',
 '东风汽车股票连续涨停东风集团首发过会对公司生产经营不构成

In [85]:
a = nlp2(list(data_list))

In [86]:
len(a)

200

In [87]:
a

[{'label': 'LABEL_1', 'score': 0.5013431310653687},
 {'label': 'LABEL_0', 'score': 0.5273839831352234},
 {'label': 'LABEL_0', 'score': 0.5132502317428589},
 {'label': 'LABEL_0', 'score': 0.5328633189201355},
 {'label': 'LABEL_1', 'score': 0.5181031823158264},
 {'label': 'LABEL_0', 'score': 0.528239905834198},
 {'label': 'LABEL_0', 'score': 0.5184571743011475},
 {'label': 'LABEL_0', 'score': 0.56069415807724},
 {'label': 'LABEL_0', 'score': 0.5500088334083557},
 {'label': 'LABEL_0', 'score': 0.5338916182518005},
 {'label': 'LABEL_0', 'score': 0.5399091243743896},
 {'label': 'LABEL_1', 'score': 0.5069075226783752},
 {'label': 'LABEL_0', 'score': 0.5402593016624451},
 {'label': 'LABEL_0', 'score': 0.547822892665863},
 {'label': 'LABEL_0', 'score': 0.5520148277282715},
 {'label': 'LABEL_0', 'score': 0.51774001121521},
 {'label': 'LABEL_0', 'score': 0.5379172563552856},
 {'label': 'LABEL_0', 'score': 0.5519105792045593},
 {'label': 'LABEL_0', 'score': 0.5037607550621033},
 {'label': 'LABEL_